There was no difference on prediction ability when using image information; will not include

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split,KFold
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier
from sklearn import preprocessing, ensemble
from sklearn.metrics import log_loss
import xgboost as xgb
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from keras.layers.core import Dense, Activation, Merge, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import Sequential
import keras
from PIL import Image
import requests
from io import BytesIO
from skimage import measure, feature, color, filters, draw, segmentation, morphology, exposure, transform
from StringIO import StringIO
from skimage import transform as tf
%matplotlib inline

stoppers=stopwords.words('english')

In [ ]:
train=pd.read_json('/Users/thomas/Data Science Projects/Two_Sigma/Files/train.json')
train=train.loc[train.index[:100]]
test=pd.read_json('/Users/thomas/Data Science Projects/Two_Sigma/Files/test.json')
test=test.loc[test.index[:5]]

In [ ]:
image_index_list=[]
image_matrix_list=[]
image_index_list_TEST=[]
image_matrix_list_TEST=[]
i=0
j=0
for row in train.index:
    j+=1
    try:
        if len(train.loc[row,'photos'])==1:
            urls=np.random.choice(train.loc[row,'photos'],1)
        else:
            urls=np.random.choice(train.loc[row,'photos'],2)
        images=[]
        for url in urls:
            response = requests.get(url)
            images.append(tf.resize(np.array(Image.open(StringIO(response.content))),(125,125,3)))
        image_matrix_list.append(images)
        image_index_list.append([row,i])
        i+=1
    except:
        print j,
        
print 
print '------'

i=0
j=0
for row in test.index:
    j+=1
    try:
        urls=np.random.choice(test.loc[row,'photos'],1)
        images=[]
        for url in urls:
            response = requests.get(url)
            images.append(tf.resize(np.array(Image.open(StringIO(response.content))),(125,125,3)))
        image_matrix_list_TEST.append(images)
        image_index_list_TEST.append([row,i])
        i+=1
    except:
        print j,

In [ ]:
def clean_street_address(address,street_block=True):
    new_address=""
    temp=address.upper()
    if "-" in temp:
        temp=temp.replace('-'," ")
    split_address=temp.split()
    if len(split_address)<1:
        return ""
    if len(split_address)==1:
        return split_address[0].upper()
    else:
        temp=[]
        for x in split_address:
            if x!="":
                temp.append(x)
        split_address=temp
        number=""
        if len(split_address)==2:
            pass
        else:
            number=split_address[0][:-2]
            split_address=split_address[1:]
        if len(split_address[0])>2:
            if (split_address[0][-2:]=='ST') or (split_address[0][-2:]=='ND') or (split_address[0][-2:]=='RD'):
                try:
                    int(split_address[0][:-2])
                    split_address[0]=split_address[0][:-2]
                except:
                    pass
        if (split_address[-1].lower()!='w') or (split_address[-1].lower()!='e'):
            if (split_address[-1][:2].lower()=='av') or (split_address[-1][:2].lower()=='st') or \
            (split_address[-1][:2].lower()=='bl') or (split_address[-1][:2].lower()=='pl'):
                split_address[-1]=split_address[-1][:2]
        
        for x in split_address:
            new_address+=x.upper()+" "
        new_address=new_address[:-1]
        new_address=new_address.replace('FIRST','1').replace('SECOND','2').replace('THIRD','3').replace('FOURTH','4')\
                    .replace('FIFTH','5').replace('SIXTH','6').replace('SEVENTH','7').replace('EIGTH','8')\
                    .replace('NINTH','9').replace('TENTH','10').replace('EAST','E').replace('WEST','W')\
                    .replace('E.','E').replace('W.','W')
        if street_block:
            if number!="":
                new_address=number+" "+new_address
        return new_address

In [ ]:
train['dayofyear']=train['created'].apply(lambda x: pd.to_datetime(x).dayofyear)
test['dayofyear']=test['created'].apply(lambda x: pd.to_datetime(x).dayofyear)
train['street_address_with_block']=train['street_address'].apply(lambda x: clean_street_address(x))
train['street_address']=train['street_address'].apply(lambda x: clean_street_address(x,street_block=False))
columns=[u'bathrooms', u'bedrooms',u'features',u'interest_level',u'price','latitude','longitude',\
      'street_address_with_block',u'street_address','dayofyear']
train=train[columns]


test['street_address_with_block']=test['street_address'].apply(lambda x: clean_street_address(x))
test['street_address']=test['street_address'].apply(lambda x: clean_street_address(x,street_block=False))

test['street_address_with_block']=test['street_address'].apply(lambda x: clean_street_address(x))
test['street_address']=test['street_address'].apply(lambda x: clean_street_address(x,street_block=False))
columns=[u'bathrooms', u'bedrooms',u'features',u'price','latitude','longitude',\
      'street_address_with_block',u'street_address','dayofyear']
test=test[columns]

In [ ]:
for row in train.index:
    features=train.loc[row,'features']
    x=""
    for feature in features:
        x+=feature.upper().replace('-',"")+" "
    x=x[:-1]
    train.loc[row,'features']=x

feature_dict={}
for row in train.index:
    for temp in train.loc[row,'features'].split():
        if temp not in stoppers:
            if temp in feature_dict.keys():
                feature_dict[temp]+=1
            else:
                feature_dict[temp]=1
                
keep_list=[]
for x in feature_dict.keys():
    if feature_dict[x]>50:
        keep_list.append(x)
        
def feature_finder(features_list,feature_to_find):
    if feature_to_find in features_list:
        return 1
    else:
        return 0

for feature in keep_list:
    train[str(feature)]=train['features'].apply(lambda x: feature_finder(x,feature))
del train['features']


# ### TESTING

for row in test.index:
    features=test.loc[row,'features']
    x=""
    for feature in features:
        x+=feature.upper().replace('-',"")+" "
    x=x[:-1]
    test.loc[row,'features']=x
        
for feature in keep_list:
    test[str(feature)]=test['features'].apply(lambda x: feature_finder(x,feature))
del test['features']

# XG BOOST

In [ ]:
# Y=train['interest_level'].copy()
# X=train.copy()
# del X['interest_level']
# X=pd.merge(X,pd.get_dummies(X['street_address_with_block']),left_index=True,right_index=True)
# X=pd.merge(X,pd.get_dummies(X[['street_address']]),left_index=True,right_index=True)
# del X['street_address']
# del X['street_address_with_block']
# # X=sparse.csr_matrix(X)

# Y=[{'high':0, 'medium':1, 'low':2}[x] for x in Y]
# x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=1)


# X_testing_set=test.copy()
# X_testing_set=pd.merge(X_testing_set,pd.get_dummies(X_testing_set['street_address_with_block']),left_index=True,right_index=True)
# X_testing_set=pd.merge(X_testing_set,pd.get_dummies(X_testing_set[['street_address']]),left_index=True,right_index=True)
# del X_testing_set['street_address']
# del X_testing_set['street_address_with_block']
# # X_testing_set=sparse.csr_matrix(X_testing_set)

In [ ]:
image_data_frame=pd.DataFrame(image_index_list,columns=['df_index','matrix_index'])
train=pd.merge(train,image_data_frame,left_index=True,right_on='df_index',how='left')

image_data_frame_TEST=pd.DataFrame(image_index_list_TEST,columns=['df_index','matrix_index'])
test=pd.merge(test,image_data_frame_TEST,left_index=True,right_on='df_index',how='left')

In [ ]:
Y=train['interest_level'].copy()
X=train.copy()
del X['interest_level']
del X['street_address']
del X['street_address_with_block']
X=X.reset_index(drop=True)
# X=sparse.csr_matrix(X)

Y=[{'high':0, 'medium':1, 'low':2}[x] for x in Y]
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=1)


X_testing_set=test.copy()
del X_testing_set['street_address']
del X_testing_set['street_address_with_block']
# X_testing_set=sparse.csr_matrix(X_testing_set)

In [ ]:
y_train_image=np.array([[Y[z],z] for z in [x for x,y in zip(x_train.index,x_train['matrix_index']) if\
                                             pd.notnull(y)]]).T
y_test_image=np.array([[Y[z],z] for z in [x for x,y in zip(x_test.index,x_test['matrix_index']) if\
                                            pd.notnull(y)]]).T
testing_set_image_index=np.array([[Y[z],z] for z in [x for x,y in \
                        zip(X_testing_set.index,X_testing_set['matrix_index']) if pd.notnull(y)]]).T

In [ ]:
x_train_image_key=x_train.loc[:,x_train.columns[-2:]]
x_test_image_key=x_test.loc[:,x_test.columns[-2:]]
x_train=x_train.loc[:,x_train.columns[:-2]]
x_test=x_test.loc[:,x_test.columns[:-2]]

x_train_images_list=[np.array(image_matrix_list[index][0]).flatten() for \
                     index in [int(x) for x in x_train_image_key['matrix_index'].values if pd.notnull(x)]]
x_test_images_list=[np.array(image_matrix_list[index][0]).flatten() for \
                    index in [int(x) for x in x_test_image_key['matrix_index'].values if pd.notnull(x)]]

#----------------------------------------
X_testing_set_image_key=X_testing_set.loc[:,X_testing_set.columns[-2:]]
X_testing_set=X_testing_set.loc[:,X_testing_set.columns[:-2]]
X_testing_set_images_list=[np.array(image_matrix_list_TEST[index][0]).flatten() for \
                     index in [int(x) for x in X_testing_set_image_key['matrix_index'].values if pd.notnull(x)]]


In [ ]:
param = {}
param['objective'] = 'multi:softprob' 
param['eta'] = 0.1 # 0.05
param['max_depth'] = 4 # 3
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 15       # 2
param['subsample'] = 0.5            # .5  
param['colsample_bytree'] = 1.      # 1.  

plst = list(param.items())

xgtrain = xgb.DMatrix(np.array(x_train_images_list), label=y_train_image[0])
xgtest = xgb.DMatrix(np.array(x_test_images_list), label=y_test_image[0])
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]

model=xgb.train(plst, xgtrain, 4000, watchlist, early_stopping_rounds=10)
preds = model.predict(xgb.DMatrix(X_testing_set_images_list))

train-mlogloss:0.739025	test-mlogloss:0.797758

In [ ]:
temp=pd.DataFrame(model.predict(xgtrain))
temp['index_lookup']=y_train_image[1]
x_train_final=pd.merge(x_train,temp,left_index=True,right_on='index_lookup', how='left')
del x_train_final['index_lookup']
x_train_final=sparse.csr_matrix(x_train_final)
#----
temp=pd.DataFrame(model.predict(xgtest))
temp['index_lookup']=y_test_image[1]
x_test_final=pd.merge(x_test,temp,left_index=True,right_on='index_lookup', how='left')
del x_test_final['index_lookup']
x_test_final=sparse.csr_matrix(x_test_final)
#----
temp=pd.DataFrame(preds)
temp['index_lookup']=testing_set_image_index[1]
X_testing_set_final=pd.merge(X_testing_set,temp,left_index=True,right_on='index_lookup', how='left')
del X_testing_set_final['index_lookup']
X_testing_set_final=sparse.csr_matrix(X_testing_set_final)

In [ ]:
param = {}
param['objective'] = 'multi:softprob' 
param['eta'] = 0.05 # 0.05
param['max_depth'] = 6 # 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 2       # 2
param['subsample'] = 0.5            # .5  
param['colsample_bytree'] = .75      # 1.  

plst = list(param.items())

xgtrain = xgb.DMatrix(x_train, label=y_train)
xgtest = xgb.DMatrix(x_test, label=y_test)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]

model=xgb.train(plst, xgtrain, 4000, watchlist, early_stopping_rounds=25)
# preds = model.predict(xgtest)

train-mlogloss:0.436267	test-mlogloss:0.615509